# Data Preprocessing

In [2]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.dates as mdates

In [3]:
data = np.load('/Volumes/MARI/ssdl_gps/correction_data/correction_data_2019_str_update.npz')
data.files

['matching_epochs',
 'matching_clock_bias',
 'matching_poly_values',
 'correction_vals']

In [6]:
epochs = data['matching_epochs']
final_bias = data['matching_clock_bias']
broadcast_bias = data['matching_poly_values']
correction = data['correction_vals']

## Removing Outliers (Considering Only Data b/w 1st and 99th Percentile)

The method described replaces values outside the 5th and 95th percentiles with the 5th or 95th percentile values, respectively. The length of data does not change; it's modifying the values within the original dataset.

Clipping: Any values below the 5th percentile are set to the 5th percentile value, and any values above the 95th percentile are set to the 95th percentile value. This means that extreme values are capped but no data is added or removed; the dataset size remains the same.

[Text partially provided from ChatGPT]

In [8]:
stacked_biases = np.column_stack((final_bias, broadcast_bias))

In [10]:
percentiles_stacked_bias_1st = np.percentile(stacked_biases, 1)
percentiles_stacked_bias_99th = np.percentile(stacked_biases, 99)

In [12]:
print(percentiles_stacked_bias_1st)
print(percentiles_stacked_bias_99th)

-0.000275395740573028
-6.418626571754036e-05


In [14]:
percentiles_correction_1st = np.percentile(correction, 1)
percentiles_correction_99th = np.percentile(correction, 99)

In [16]:
print(percentiles_correction_1st)
print(percentiles_correction_99th)

-1.3656707763352177e-09
3.6908192275824645e-09


In [18]:
clipped_stacked_bias = np.clip(stacked_biases, percentiles_stacked_bias_1st, percentiles_stacked_bias_99th)
clipped_correction = np.clip(correction, percentiles_correction_1st, percentiles_correction_99th)

In [20]:
clipped_final_bias = clipped_stacked_bias[:, 0]
clipped_broadcast_bias = clipped_stacked_bias[:, 1]

In [22]:
np.savez('/Volumes/MARI/ssdl_gps/correction_data/correction_data_2019_outliers_1_99.npz',
         matching_epochs=epochs,
         matching_clock_bias=clipped_final_bias,
         matching_poly_values=clipped_broadcast_bias,
         correction_vals=clipped_correction)

In [24]:
print(f'Maximum Clipped Final Bias: {max(clipped_final_bias)}')
print(f'Minimum Clipped Final Bias: {min(clipped_final_bias)}')
print(f'Maximum Clipped Broadcast Bias: {max(clipped_broadcast_bias)}')
print(f'Minimum Clipped Broadcast Bias: {min(clipped_broadcast_bias)}')
print(f'Maximum Clipped Correction: {max(clipped_correction)}')
print(f'Minimum Clipped Correction: {min(clipped_correction)}')

Maximum Clipped Final Bias: -6.418626571754036e-05
Minimum Clipped Final Bias: -0.000275395740573028
Maximum Clipped Broadcast Bias: -6.418626571754036e-05
Minimum Clipped Broadcast Bias: -0.000275395740573028
Maximum Clipped Correction: 3.6908192275824645e-09
Minimum Clipped Correction: -1.3656707763352177e-09


In [26]:
print(f'Maximum Final Bias: {max(final_bias)}')
print(f'Minimum Final Bias: {min(final_bias)}')
print(f'Maximum Broadcast Bias: {max(broadcast_bias)}')
print(f'Minimum Broadcast Bias: {min(broadcast_bias)}')
print(f'Maximum Correction: {max(correction)}')
print(f'Minimum Correction: {min(correction)}')

Maximum Final Bias: -6.24509320829e-05
Minimum Final Bias: -0.000278481206632
Maximum Broadcast Bias: -6.24519193479468e-05
Minimum Broadcast Bias: -0.0002776021137834
Maximum Correction: 5.931226931664034e-09
Minimum Correction: -4.122253926605666e-06


## Plotting Data After Outlier Processing

In [28]:
time_datetimes = [datetime.strptime(ts, '%Y:%m:%d:%H:%M:%S') for ts in epochs]

In [30]:
for month in range(1, 13):
    start_date = datetime(2019, month, 1)
    if month == 12:
        end_date = datetime(2020, 1, 1)
    else:
        end_date = datetime(2019, month + 1, 1)

    monthly_indices = [i for i, epoch in enumerate(time_datetimes) if start_date <= epoch < end_date]
    monthly_epochs = [time_datetimes[i] for i in monthly_indices]
    monthly_clock_bias = [clipped_final_bias[i] for i in monthly_indices]
    monthly_poly_values = [clipped_broadcast_bias[i] for i in monthly_indices]
    monthly_correction_vals = [clipped_correction[i] for i in monthly_indices]

    # Plot clock_bias and poly_values
    plt.figure(figsize=(50, 10))
    plt.scatter(monthly_epochs, monthly_clock_bias, label='Clock Bias (Station: GRG)')
    plt.scatter(monthly_epochs, monthly_poly_values, label='Broadcast Polynomial Values', s=5)
    plt.xlabel('Time (YYYY:MM:DD:HH:MI:SS)')
    plt.ylabel('Bias Values (s)')
    # plt.ylim(-0.00002734, -0.00002722)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=5))  # Set major ticks every hour
    # plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.grid(color='gray', linestyle='--', linewidth=0.25)
    plt.xticks(rotation=90)
    plt.legend()
    plt.title(f'Clock Bias and Poly Values for G21 {start_date.strftime("%B %Y")}')
    plt.savefig(f'/Volumes/MARI/ssdl_gps/plots_outlier_1_99/clock_bias_poly_values_{month:02d}_2019.png')
    plt.close()
    
    # Plot correction values
    plt.figure(figsize=(50, 10))
    plt.scatter(monthly_epochs, monthly_correction_vals, label='Clock Bias Correction (s)')
    plt.xlabel('Time (YYYY:MM:DD:HH:MI:SS)')
    plt.ylabel('Bias Correction Values (s)')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=5))  # Set major ticks every hour
    # plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.grid(color='gray', linestyle='--', linewidth=0.25)
    plt.xticks(rotation=90)
    plt.legend()
    plt.title(f'Clock Bias Correction Values for G21 {start_date.strftime("%B %Y")}')
    plt.savefig(f'/Volumes/MARI/ssdl_gps/plots_outlier_1_99/correction_values_{month:02d}_2019.png')
    plt.close() 

## Min-Max Scaling

In [32]:
scaler = MinMaxScaler()

In [34]:
scaled_biases = scaler.fit_transform(clipped_stacked_bias)

In [36]:
scaled_final_bias = scaled_biases[:, 0]
scaled_broadcast_bias = scaled_biases[:, 1]

In [38]:
correction_scaler = MinMaxScaler()

In [42]:
scaled_correction = correction_scaler.fit_transform(clipped_correction.reshape(-1, 1)).flatten()

In [47]:
np.savez('/Volumes/MARI/ssdl_gps/correction_data/correction_data_2019_minmaxscale_postoutlier.npz',
         matching_epochs=epochs,
         matching_clock_bias=scaled_final_bias,
         matching_poly_values=scaled_broadcast_bias,
         correction_vals=scaled_correction)

In [51]:
print(f'Maximum Scaled Final Bias: {max(scaled_final_bias)}')
print(f'Minimum Scaled Final Bias: {min(scaled_final_bias)}')
print(f'Maximum Scaled Broadcast Bias: {max(scaled_broadcast_bias)}')
print(f'Minimum Scaled Broadcast Bias: {min(scaled_broadcast_bias)}')
print(f'Maximum Scaled Correction: {max(scaled_correction)}')
print(f'Minimum Scaled Correction: {min(scaled_correction)}')

Maximum Scaled Final Bias: 0.9999999999999999
Minimum Scaled Final Bias: 0.0
Maximum Scaled Broadcast Bias: 0.9999999999999999
Minimum Scaled Broadcast Bias: 0.0
Maximum Scaled Correction: 0.9999999999999998
Minimum Scaled Correction: 0.0


In [44]:
for month in range(1, 13):
    start_date = datetime(2019, month, 1)
    if month == 12:
        end_date = datetime(2020, 1, 1)
    else:
        end_date = datetime(2019, month + 1, 1)

    monthly_indices = [i for i, epoch in enumerate(time_datetimes) if start_date <= epoch < end_date]
    monthly_epochs = [time_datetimes[i] for i in monthly_indices]
    monthly_clock_bias = [scaled_final_bias[i] for i in monthly_indices]
    monthly_poly_values = [scaled_broadcast_bias[i] for i in monthly_indices]
    monthly_correction_vals = [scaled_correction[i] for i in monthly_indices]

    # Plot clock_bias and poly_values
    plt.figure(figsize=(50, 10))
    plt.scatter(monthly_epochs, monthly_clock_bias, label='Clock Bias (Station: GRG)')
    plt.scatter(monthly_epochs, monthly_poly_values, label='Broadcast Polynomial Values', s=5)
    plt.xlabel('Time (YYYY:MM:DD:HH:MI:SS)')
    plt.ylabel('Bias Values (s)')
    # plt.ylim(-0.00002734, -0.00002722)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=5))  # Set major ticks every hour
    # plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.grid(color='gray', linestyle='--', linewidth=0.25)
    plt.xticks(rotation=90)
    plt.legend()
    plt.title(f'Clock Bias and Poly Values for G21 {start_date.strftime("%B %Y")}')
    plt.savefig(f'/Volumes/MARI/ssdl_gps/plots_minmaxscaler/clock_bias_poly_values_{month:02d}_2019.png')
    plt.close()
    
    # Plot correction values
    plt.figure(figsize=(50, 10))
    plt.scatter(monthly_epochs, monthly_correction_vals, label='Clock Bias Correction (s)')
    plt.xlabel('Time (YYYY:MM:DD:HH:MI:SS)')
    plt.ylabel('Bias Correction Values (s)')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=5))  # Set major ticks every hour
    # plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.grid(color='gray', linestyle='--', linewidth=0.25)
    plt.xticks(rotation=90)
    plt.legend()
    plt.title(f'Clock Bias Correction Values for G21 {start_date.strftime("%B %Y")}')
    plt.savefig(f'/Volumes/MARI/ssdl_gps/plots_minmaxscaler/correction_values_{month:02d}_2019.png')
    plt.close() 

# Model 

In [ ]:
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
# import numpy as np
# import torch
# from sklearn.preprocessing import MinMaxScaler
# from torch.utils.data import TensorDataset

# # Load data
# data = np.load('/Volumes/MARI/ssdl_gps/correction_data_2019_mo1.npz')

# matching_poly_values = data['matching_poly_values']
# correction_vals = data['correction_vals']

# # Ensure data is in the correct shape [num_samples, num_features]
# # Assuming each sample has one feature (i.e., 1D data)
# matching_poly_values = matching_poly_values[:, np.newaxis]  # Shape: [num_samples, 1]
# correction_vals = correction_vals[:, np.newaxis]  # Shape: [num_samples, 1]

# # Apply min-max scaling
# scaler_poly = MinMaxScaler()
# scaler_correction = MinMaxScaler()

# matching_poly_values_scaled = scaler_poly.fit_transform(matching_poly_values)
# correction_vals_scaled = scaler_correction.fit_transform(correction_vals)

# # Split data into train and test sets
# train_size = int(0.7 * len(matching_poly_values_scaled))
# test_size = len(matching_poly_values_scaled) - train_size

# train_poly_values = matching_poly_values_scaled[:train_size]
# train_correction_vals = correction_vals_scaled[:train_size]

# test_poly_values = matching_poly_values_scaled[train_size:]
# test_correction_vals = correction_vals_scaled[train_size:]

# # Convert to torch tensors
# train_poly_values = torch.from_numpy(train_poly_values).float()
# train_correction_vals = torch.from_numpy(train_correction_vals).float()
# test_poly_values = torch.from_numpy(test_poly_values).float()
# test_correction_vals = torch.from_numpy(test_correction_vals).float()

# # Create datasets
# train_dataset = TensorDataset(train_poly_values, train_correction_vals)
# test_dataset = TensorDataset(test_poly_values, test_correction_vals)

# Test Code

In [ ]:
data = np.load('/Volumes/MARI/ssdl_gps/correction_data/correction_data_2019_mo1.npz')
data.files

In [ ]:
np.savez('test_correction_data_2019_mo1.npz',
         correction_vals = data['correction_vals'])

In [ ]:
x = range(len(data['correction_vals']))
plt.scatter(x,data['correction_vals'])

# Transformer Code

In [ ]:
# np.random.seed(3)

# def generate_data(data_path: Path, num_steps: int = 1000000, interval: float = 0.1) -> None:
#     """
#     Generate synthetic data and save it to a specified file.

#     Parameters ----------------------------------------------------------------
#     data_path : Path
#         The path where the generated data will be saved.
#     num_steps : int, optional
#         The number of data points to generate.
#     interval : float, optional
#         The spacing between data points in the x-axis (default is 0.1).

#     Returns --------------------------------------------------------------------
#     -------
#     None
#         The function saves the generated data to a file at `data_path`.
#     """
#     x = np.linspace(0, num_steps * interval, num_steps)
#     y = np.sin(x) + np.random.normal(0, 0.1, x.shape)

#     np.savez(data_path, y=y)

#     return num_steps, interval

In [ ]:
# sine = Path("sine_data.npz")
# sawtooth = Path("saw_data.npz")

In [ ]:
# np.random.seed(0)

# def generate_data(data_path: Path, num_steps: int = 1000000, interval: float = 0.1, waveform_type: str = 'sine') -> None:
#     """
#     Generate synthetic data (sine or noisy sawtooth wave) and save it to a specified file.

#     Parameters
#     ----------
#     data_path : Path
#         The path where the generated data will be saved.
#     num_steps : int, optional
#         The number of data points to generate.
#     interval : float, optional
#         The spacing between data points in the x-axis (default is 0.1).
#     waveform_type : str, optional
#         Type of waveform to generate ('sine' or 'sawtooth'). Default is 'sine'.

#     Returns
#     -------
#     None
#         The function saves the generated data to a file at `data_path`.
#     """
#     if waveform_type == 'sine':
#         x = np.linspace(0, num_steps * interval, num_steps)
#         y = np.sin(x) + np.random.normal(0, 0.1, x.shape)
#     elif waveform_type == 'sawtooth':
#         sampling_rate = 4
#         duration = 100000
#         x = np.linspace(0.0, duration, int(duration * sampling_rate), endpoint=False)
#         amplitude = 0.5  # Amplitude of the sawtooth wave
#         noise_amplitude = 0.1  # Amplitude of the noise
#         frequency = .04  # Frequency of the sawtooth wave in Hz

#         # Sawtooth wave
#         sawtooth_wave = amplitude * (2 * (frequency * x - np.floor(frequency * x + 0.5)))
#         noise = noise_amplitude * np.random.randn(len(x))
#         y = sawtooth_wave + noise
#     else:
#         raise ValueError("Unsupported waveform_type. Choose 'sine' or 'sawtooth'.")

#     np.savez(data_path, y=y)

#     return x, num_steps, interval

In [ ]:
# (x_sine, num_steps, interval) = generate_data(sine, num_steps=1000000, interval=0.1, waveform_type='sine')
# (x_saw, num_steps, interval) = generate_data(sawtooth, num_steps=1000000, interval=0.1, waveform_type='sawtooth')

In [ ]:
# plt.plot(x_sine, np.load('sine_data.npz')['y'], color='r', label='Noisy Sine Wave')
# plt.grid(True)
# plt.xlim(0,100)
# plt.show()

In [ ]:
# plt.plot(x_saw, np.load('saw_data.npz')['y'], color='r', label='Noisy Sawtooth Wave')
# plt.grid(True)
# plt.xlim(0,100)
# plt.show()

In [ ]:
def split_sequence(
    sequence: np.ndarray, ratio: float = 0.8
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """Splits a sequence into 2 (3) parts, as is required by our transformer
    model.

    Assume our sequence length is L, we then split this into src of length N
    and tgt_y of length M, with N + M = L.
    src, the first part of the input sequence, is the input to the encoder, and we
    expect the decoder to predict tgt_y, the second part of the input sequence.
    In addition we generate tgt, which is tgt_y but "shifted left" by one - i.e. it
    starts with the last token of src, and ends with the second-last token in tgt_y.
    This sequence will be the input to the decoder.


    Args:

        sequence: batched input sequences to split [bs, seq_len, num_features]
        ratio: split ratio, N = ratio * L

    Returns:
        tuple[torch.Tensor, torch.Tensor, torch.Tensor]: src, tgt, tgt_y
    """
    src_end = int(sequence.shape[1] * ratio)
    # [bs, src_seq_len, num_features]
    src = sequence[:, :src_end]
    # [bs, tgt_seq_len, num_features]
    tgt = sequence[:, src_end - 1 : -1]
    # [bs, tgt_seq_len, num_features]
    tgt_y = sequence[:, src_end:]

    return src, tgt, tgt_y

In [ ]:
# Taken from https://pytorch.org/tutorials/beginner/transformer_tutorial.html,
# only modified to account for "batch first"

class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000) -> None:
      """
      Args:
        d_model (int): Dimension of the model (embedding dimension)
        dropout (float, optional): Dropout probability. Default is 0.1.
        max_len (int, optional): Maximum length of input sequences. Default is 5000.

      Attributes:
        pe (torch.Tensor): Positional encoding tensor. Shape: (1, max_len, d_model)

      Returns:
        torch.Tensor: input tensor with added positional encoding.

      """
      super().__init__()
      self.dropout = torch.nn.Dropout(p=dropout)

      position = torch.arange(max_len).unsqueeze(1)                           # 1-D tensor from 0 to max_len -1. Unsqueeze "adds" a superficial 1 dim.
      div_term = torch.exp(
          torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
      )
      pe = torch.zeros(1, max_len, d_model)
      pe[0, :, 0::2] = torch.sin(position * div_term)
      pe[0, :, 1::2] = torch.cos(position * div_term)
      self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
      """Adds positional encoding to the given tensor.

      Args:
          x: tensor to add PE to [bs, seq_len, embed_dim]

      Returns:
          torch.Tensor: tensor with PE [bs, seq_len, embed_dim]
      """
      x = x + self.pe[:, : x.size(1)]
      return self.dropout(x)

In [ ]:
class TransformerWithPE(torch.nn.Module):
    def __init__(
        self, in_dim: int, out_dim: int, embed_dim: int, num_heads: int, num_layers: int
    ) -> None:
        """
        Initializes a transformer model with positional encoding.

        Args:
            in_dim: number of input features
            out_dim: number of features to predict
            embed_dim: embed features to this dimension
            num_heads: number of transformer heads
            num_layers: number of encoder and decoder layers
        """
        super().__init__()

        self.positional_encoding = PositionalEncoding(embed_dim)

        # transform input features into embedded features
        self.encoder_embedding = torch.nn.Linear(
            in_features=in_dim, out_features=embed_dim
        )
        self.decoder_embedding = torch.nn.Linear(
            in_features=out_dim, out_features=embed_dim
        )

        # map output into output dimension
        self.output_layer = torch.nn.Linear(in_features=embed_dim, out_features=out_dim)


        self.transformer = torch.nn.Transformer(
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            d_model=embed_dim,
            batch_first=True,
        )

    def forward(self, src: torch.Tensor, tgt: torch.Tensor) -> torch.Tensor:
        """Forward function of the model.

        Args:
            src: input sequence to the encoder [bs, src_seq_len, num_features]
            tgt: input sequence to the decoder [bs, tgt_seq_len, num_features]

        Returns:
            torch.Tensor: predicted sequence [bs, tgt_seq_len, feat_dim]
        """
        # if self.train:
        # Add noise to decoder inputs during training
        # tgt = tgt + torch.normal(0, 0.1, size=tgt.shape).to(tgt.device)

        # Embed encoder input and add positional encoding.
        # [bs, src_seq_len, embed_dim]
        src = self.encoder_embedding(src)
        src = self.positional_encoding(src)

        # Generate mask to avoid attention to future outputs.
        # [tgt_seq_len, tgt_seq_len]
        tgt_mask = torch.nn.Transformer.generate_square_subsequent_mask(tgt.shape[1])
        # Embed decoder input and add positional encoding.
        # [bs, tgt_seq_len, embed_dim]
        tgt = self.decoder_embedding(tgt)
        tgt = self.positional_encoding(tgt)

        # Get prediction from transformer and map to output dimension.
        # [bs, tgt_seq_len, embed_dim]
        pred = self.transformer(src, tgt, tgt_mask=tgt_mask)
        pred = self.output_layer(pred)

        return pred                                                             # return predicted sequence


    def infer(self, src: torch.Tensor, tgt_len: int) -> torch.Tensor:
        """Runs inference with the model, meaning: predicts future values
        for an unknown sequence.
        For this, iteratively generate the next output token while
        feeding the already generated ones as input sequence to the decoder.

        Args:
            src: input to the encoder [bs, src_seq_len, num_features]
            tgt_len: desired length of the output

        Returns:
            torch.Tensor: inferred sequence
        """
        output = torch.zeros((src.shape[0], tgt_len + 1, src.shape[2])).to(src.device)
        output[:, 0] = src[:, -1]
        for i in range(tgt_len):
            output[:, i + 1] = self.forward(src, output)[:, i]

        return output[:, 1:]

In [ ]:
def load_and_partition_data(
    data_path: Path, seq_length: int = 100
) -> tuple[np.ndarray, int]:
    """Loads the given data and paritions it into sequences of equal length.

    Args:
        data_path: path to the dataset
        sequence_length: length of the generated sequences

    Returns:
        tuple[np.ndarray, int]: tuple of generated sequences and number of
            features in dataset
    """
    data = np.load(data_path)
    num_features = len(data.keys())

    # Check that each feature provides the same number of data points
    data_lens = [len(data[key]) for key in data.keys()]
    assert len(set(data_lens)) == 1

    num_sequences = data_lens[0] // seq_length
    sequences = np.empty((num_sequences, seq_length, num_features))

    for i in range(0, num_sequences):
        # [sequence_length, num_features]
        sample = np.asarray(
            [data[key][i * seq_length : (i + 1) * seq_length] for key in data.keys()]
        ).swapaxes(0, 1)
        sequences[i] = sample

    return sequences, num_features


def make_datasets(sequences: np.ndarray) -> tuple[TensorDataset, TensorDataset]:
    """Create train and test dataset.

    Args:
        sequences: sequences to use [num_sequences, sequence_length, num_features]

    Returns:
        tuple[TensorDataset, TensorDataset]: train and test dataset
    """
    # Split sequences into train and test split
    train, test = train_test_split(sequences, test_size=0.2)
    return TensorDataset(torch.Tensor(train)), TensorDataset(torch.Tensor(test))


def visualize(
    src: torch.Tensor,
    tgt: torch.Tensor,
    pred: torch.Tensor,
    pred_infer: torch.Tensor,
    idx=0,
) -> None:
    """Visualizes a given sample including predictions.

    Args:
        src: source sequence [bs, src_seq_len, num_features]
        tgt: target sequence [bs, tgt_seq_len, num_features]
        pred: prediction of the model [bs, tgt_seq_len, num_features]
        pred_infer: prediction obtained by running inference
            [bs, tgt_seq_len, num_features]
        idx: batch index to visualize
    """
    x = np.arange(src.shape[1] + tgt.shape[1])
    src_len = src.shape[1]

    plt.plot(x[:src_len], src[idx].cpu().detach(), "bo-", label="src")
    plt.plot(x[src_len:], tgt[idx].cpu().detach(), "go-", label="tgt")
    plt.plot(x[src_len:], pred[idx].cpu().detach(), "ro-", label="pred")
    plt.plot(x[src_len:], pred_infer[idx].cpu().detach(), "yo-", label="pred_infer")

    plt.legend()
    plt.show()
    plt.clf()


def split_sequence(
    sequence: np.ndarray, ratio: float = 0.8
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """Splits a sequence into 2 (3) parts, as is required by our transformer
    model.

    Assume our sequence length is L, we then split this into src of length N
    and tgt_y of length M, with N + M = L.
    src, the first part of the input sequence, is the input to the encoder, and we
    expect the decoder to predict tgt_y, the second part of the input sequence.
    In addition we generate tgt, which is tgt_y but "shifted left" by one - i.e. it
    starts with the last token of src, and ends with the second-last token in tgt_y.
    This sequence will be the input to the decoder.


    Args:
        sequence: batched input sequences to split [bs, seq_len, num_features]
        ratio: split ratio, N = ratio * L

    Returns:
        tuple[torch.Tensor, torch.Tensor, torch.Tensor]: src, tgt, tgt_y
    """
    src_end = int(sequence.shape[1] * ratio)
    # [bs, src_seq_len, num_features]
    src = sequence[:, :src_end]
    # [bs, tgt_seq_len, num_features]
    tgt = sequence[:, src_end - 1 : -1]
    # [bs, tgt_seq_len, num_features]
    tgt_y = sequence[:, src_end:]

    return src, tgt, tgt_y


def move_to_device(device: torch.Tensor, *tensors: torch.Tensor) -> list[torch.Tensor]:
    """Move all given tensors to the given device.

    Args:
        device: device to move tensors to
        tensors: tensors to move

    Returns:
        list[torch.Tensor]: moved tensors
    """
    moved_tensors = []
    for tensor in tensors:
        if isinstance(tensor, torch.Tensor):
            moved_tensors.append(tensor.to(device))
        else:
            moved_tensors.append(tensor)
    return moved_tensors

In [ ]:
BS = 100                                                                        # batch size
FEATURE_DIM = 128                                                               # dimensionality of input features
NUM_HEADS = 8                                                                   # number of attention heads in the multi-head attention mechanism
NUM_EPOCHS = 10                                                                 # number of times entire dataset is passed for training
NUM_VIS_EXAMPLES = 1
NUM_LAYERS = 2                                                                  # number of encoder and decoder layers in the mdel
LR = 0.001  

In [ ]:
# Load data and generate train and test datasets / dataloaders
sequences, num_features = load_and_partition_data("test_correction_data_2019_mo1.npz")               # change data file name
train_set, test_set = make_datasets(sequences)
train_loader, test_loader = DataLoader(
    train_set, batch_size=BS, shuffle=True
), DataLoader(test_set, batch_size=BS, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Initialize model, optimizer and loss criterion
model = TransformerWithPE(
    num_features, num_features, FEATURE_DIM, NUM_HEADS, NUM_LAYERS
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = torch.nn.MSELoss()

In [ ]:
%%time

# Train loop
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0                                                            # initialize epoch loss
    for batch in train_loader:
        optimizer.zero_grad()

        src, tgt, tgt_y = split_sequence(batch[0])
        src, tgt, tgt_y = move_to_device(device, src, tgt, tgt_y)
        # [bs, tgt_seq_len, num_features]
        pred = model(src, tgt)
        loss = criterion(pred, tgt_y)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(
        f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Loss: "
        f"{(epoch_loss / len(train_loader)):.4f}"
    )

In [ ]:
%%time
# Evaluate model
model.eval()
eval_loss = 0.0
infer_loss = 0.0

with torch.no_grad():
    for idx, batch in enumerate(test_loader):
        src, tgt, tgt_y = split_sequence(batch[0])
        src, tgt, tgt_y = move_to_device(device, src, tgt, tgt_y)

        # [bs, tgt_seq_len, num_features]
        pred = model(src, tgt)
        loss = criterion(pred, tgt_y)
        eval_loss += loss.item()

        # Run inference with model
        pred_infer = model.infer(src, tgt.shape[1])
        loss_infer = criterion(pred_infer, tgt_y)
        infer_loss += loss_infer.item()

        if idx < NUM_VIS_EXAMPLES:
            visualize(src, tgt, pred, pred_infer)

avg_eval_loss = eval_loss / len(test_loader)
avg_infer_loss = infer_loss / len(test_loader)

print(f"Eval Loss on test set: {avg_eval_loss:.4f}")